In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb
    

In [2]:
train = pd.read_csv('train.csv')
benchmark = pd.read_csv('benchmark.csv')
prev = pd.read_csv('prev.csv')

X = train.drop(labels = ['LABEL'], axis=1)
X = pd.get_dummies(X, columns=['MEDICAL_UNIT'])

y = np.array(train['LABEL'])
y_encoded = LabelEncoder().fit(y).transform(y)
y_dummies = pd.get_dummies(y_encoded)

prev = prev.drop(labels = ['ID'], axis=1)
prev = pd.get_dummies(prev, columns=['MEDICAL_UNIT'])
prev.insert(X.columns.get_loc('MEDICAL_UNIT_2'), 'MEDICAL_UNIT_2', np.zeros(len(prev)).astype(int))





X_train, X_test, y_train_dummies, y_test_dummies = train_test_split(X, y_dummies, test_size=0.33, random_state=0)

y_test_encoded = np.ravel(pd.from_dummies(y_test_dummies))
y_test = LabelEncoder().fit(y).inverse_transform(y_test_encoded.tolist())

y_train_encoded = np.ravel(pd.from_dummies(y_train_dummies))
y_train = LabelEncoder().fit(y).inverse_transform(y_train_encoded.tolist())

prev

,USMER,SEX,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,...,MEDICAL_UNIT_4,MEDICAL_UNIT_5,MEDICAL_UNIT_6,MEDICAL_UNIT_7,MEDICAL_UNIT_8,MEDICAL_UNIT_9,MEDICAL_UNIT_10,MEDICAL_UNIT_11,MEDICAL_UNIT_12,MEDICAL_UNIT_13
0,1,2,1,73,2,1,2,2,2,1,...,1,0,0,0,0,0,0,0,0,0
1,2,1,2,39,2,2,2,1,2,2,...,0,0,0,0,0,0,0,0,1,0
2,1,2,1,46,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,1,0
3,1,1,1,74,2,2,2,2,2,1,...,0,0,0,0,0,0,0,0,1,0
4,1,2,1,1,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7507,2,1,1,52,2,1,2,2,2,1,...,0,0,0,0,0,0,0,0,1,0
7508,2,1,1,51,2,2,2,2,2,2,...,0,0,0,0,0,1,0,0,0,0
7509,2,1,1,66,2,2,1,2,2,2,...,0,0,0,0,0,0,0,0,1,0
7510,2,2,2,62,2,2,2,2,2,1,...,1,0,0,0,0,0,0,0,0,0


# BOOTSTRAPING

# XGBRegressor

# Model 1 - Random state : 42 - Not encoded

In [3]:
my_model1 = xgb.XGBRegressor(
                        n_estimators=200, 
                        learning_rate=0.044, 
                        n_jobs=0, 
                        gamma=0, 
                        max_depth=5,
                        min_child_weight=30, 
                        max_delta_step=0.25,
                        subsample=1, 
                        num_parallel_tree=20, #set to 20 for actual prediction
                        )

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    
    my_model1.fit(X_train, y_train_dummies,
                 early_stopping_rounds=5, 
                 eval_set=[(X_test, y_test_dummies)], 
                 verbose=0)



predictions_dummies = my_model1.predict(X_test)
predictions_encoded = [np.argmax(i) for i in predictions_dummies]
predictions = LabelEncoder().fit(y).inverse_transform(predictions_encoded)
predictions

best1 = accuracy_score(predictions, y_test)

print(accuracy_score(predictions, y_test))
print(accuracy_score(predictions, y_test)>best1)

0.6833719874962186
False


In [4]:
best1

0.6833719874962186

# Model 2 - Random State : 0 - Not encoded

In [5]:
my_model2 = xgb.XGBRegressor(
                        n_estimators=95, 
                        learning_rate=0.044, 
                        n_jobs=0, 
                        gamma=1.02, 
                        max_depth=5,
                        min_child_weight=39, 
                        max_delta_step=0.25,
                        num_parallel_tree=20, #set to 20 for actual prediction
                        )

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    
    my_model2.fit(X_train, y_train_dummies,
                 early_stopping_rounds=5, 
                 eval_set=[(X_test, y_test_dummies)], 
                 verbose=0)



predictions_dummies = my_model2.predict(X_test)
predictions_encoded = [np.argmax(i) for i in predictions_dummies]
predictions = LabelEncoder().fit(y).inverse_transform(predictions_encoded)
predictions

best2 = accuracy_score(predictions, y_test)

print(accuracy_score(predictions, y_test))
print(accuracy_score(predictions, y_test)>=best2)

0.6812544116164162
True


In [6]:

best2

0.6812544116164162

# Model 3 - Random State : 0 - Encoded

In [7]:
my_model3 = xgb.XGBRegressor(
                        n_estimators=140, 
                        learning_rate=0.044, 
                        n_jobs=0, 
                        gamma=0, 
                        max_depth=4,
                        min_child_weight=11, 
                        max_delta_step=0.25,
                        subsample=1, 
                        num_parallel_tree=1, #set to 1 for actual prediction
                        )

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    
    my_model3.fit(X_train, y_train_dummies,
                 early_stopping_rounds=10,
                 eval_set=[(X_test, y_test_dummies)], 
                 eval_metric="error",
                 verbose=0)



predictions_dummies = my_model3.predict(X_test)
predictions_encoded = [np.argmax(i) for i in predictions_dummies]
predictions = LabelEncoder().fit(y).inverse_transform(predictions_encoded)
predictions

best3 = accuracy_score(predictions, y_test)

print(accuracy_score(predictions, y_test))
print(accuracy_score(predictions, y_test)>=best3)

0.6858929111626499
True


In [8]:
best3 = accuracy_score(predictions, y_test)
best3

0.6858929111626499

In [9]:
from pandas_ml import ConfusionMatrix

confusion_matrix = ConfusionMatrix(predictions, y_test)
confusion_matrix.print_stats()

AttributeError: module 'sklearn.metrics' has no attribute 'jaccard_similarity_score'

# Model 4 - Random State : 0 - Encoded - Classifier

In [10]:
my_model4 = xgb.XGBClassifier() #objective="multi:softmax"

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    
    my_model4.fit(X_train, y_train_dummies,
                 early_stopping_rounds=5, 
                 eval_set=[(X_test, y_test_dummies)], 
                 eval_metric="merror",
                 verbose=0)



predictions_dummies = my_model3.predict(X_test)
predictions_encoded = [np.argmax(i) for i in predictions_dummies]
predictions = LabelEncoder().fit(y).inverse_transform(predictions_encoded)
predictions

print(accuracy_score(predictions, y_test))
#print(accuracy_score(predictions, y_test)>=best3)



XGBoostError: [20:48:16] c:\users\dev-admin\croot2\xgboost-split_1675461376218\work\src\metric\multiclass_metric.cu:34: Check failed: label_error >= 0 && label_error < static_cast<int32_t>(n_class): MultiClassEvaluation: label must be in [0, num_class), num_class=1 but found 1 in label

# Prev prediction

In [9]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    
    my_model3.fit(X_train, y_train_dummies,
                 early_stopping_rounds=5, 
                 eval_set=[(X_test, y_test_dummies)], 
                 verbose=0)
    


In [10]:
prev_predictions_dummies = my_model3.predict(prev)
prev_predictions_encoded = [np.argmax(i) for i in prev_predictions_dummies]
prev_predictions = LabelEncoder().fit(y).inverse_transform(prev_predictions_encoded)

len(prev_predictions)
prev_predictions

array(['DEAD', 'SOFT_COVID', 'STRONG_COVID', ..., 'DEAD', 'DEAD',
       'STRONG_COVID'], dtype=object)

In [11]:
prev_predictions = pd.DataFrame(prev_predictions)
prev_predictions = prev_predictions.rename(columns={0 : 'LABEL'})
prev_predictions

,LABEL
0,DEAD
1,SOFT_COVID
2,STRONG_COVID
3,DEAD
4,STRONG_COVID
...,...
7507,STRONG_COVID
7508,STRONG_COVID
7509,DEAD
7510,DEAD


In [14]:
prev_predictions.to_csv(r'C:\Users\caohu\Desktop\ARE SDD\XBG_Predictions5_ARESD_Thierry.csv', \
             index=True, header=['LABEL'], index_label='ID')

OSError: Cannot save file into a non-existent directory: 'C:\Users\caohu\Desktop\ARE SDD'

# Experimental features

In [15]:
preprocessor = Pipeline(steps=[('label_encoder', LabelEncoder()),
                               #('dummies', pd.get_dummies(y_encoded))
                              ])

#y_encoded = LabelEncoder().fit(y).transform(y)
#y_dummies = pd.get_dummies(y_encoded)

preprocessor.fit(y_train)
#.transform(y)

TypeError: fit() takes 2 positional arguments but 3 were given

In [16]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model),
                              ('postprocessor', postprocessor),
                             ])

NameError: name 'my_model' is not defined

In [17]:
scores = -1 * cross_val_score(my_model, 
                              X_train, y_train_dummies,
                              cv=5,
                              scoring='accuracy')

scores

NameError: name 'my_model' is not defined

# KNN 

In [5]:


# Instantiate a KNN classifier object
knn = KNeighborsClassifier(n_neighbors=10)

# Train the classifier on the training data
knn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn.predict(X_test)

# Evaluate the performance of the KNN classifier
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)
#pred={
#    'LABEL':y_pred
#}
#submision=pd.DataFrame(pred)
#submision.to_csv("submision2.csv",index="ID")

0.6400121004335989


# Node class


In [7]:
class Node():
    def __init__(self,feature_index=None,threshold=None,left=None,right=None,info_gain=None, value=None):
        #decision node
        self.feature_index=feature_index
        self.threshold=threshold
        self.left=left
        self.right=right
        self.info_gain=info_gain
        #Leaf node
        self.value=value

# Tree class

In [15]:
class Decision_tree():
    def __init__(self,min_samples_split=2,max_depth=2):
       #initialize the root
        self.root=None
        
        #stopping conditions
        self.min_samples_split= min_samples_split
        self.max_depth=max_depth
        
    def build_tree(self,dataset,curr_depth=0):
        X,Y= dataset[:,:-1],dataset[:,-1]
        nb_samples,nb_features=np.shape(X)
        
        #split until conditions are met

        if nb_samples>=self.min_samples_split and curr_depth<self.max_depth:
            
            best_split=self.get_best_split(dataset, nb_samples,nb_features)
            #print(best_split)
            if best_split!={}:
                if best_split['info_gain']>0  :
                    #print(best_split['info_gain'])
                    left_subtree=self.build_tree(best_split["dataset_left"],curr_depth+1)
                    rigth_subtree=self.build_tree(best_split["dataset_right"],curr_depth+1)
                    return Node(best_split["feature_index"],best_split["threshold"],
                            left_subtree,rigth_subtree,best_split["info_gain"])
     
        #compute leaf node
        leaf_value=self.calculate_leaf_value(Y)
        #print(leaf_value)
        return Node(value=leaf_value)
        
        
    def get_best_split(self,dataset,nb_samples,nb_features):
            
            #dictionary to store the best split
            best_split={}
            max_info_gain=-float("inf")
            
            #loop over all the features
            for feature_index in range(nb_features):
                feature_values=dataset[:,feature_index]
                possible_thresholds=np.unique(feature_values)
                #loop over all the feature values present in data
                for threshold in possible_thresholds:
                    #get current split
                    dataset_left,dataset_right= self.split(dataset,feature_index,threshold)
                    if len(dataset_left)>0 and len(dataset_right)>0:
                        y,left_y,right_y=dataset[:,-1],dataset_left[:,-1],dataset_right[:,-1]
                        #compute info gain
                        curr_info_gain=self.information_gain(y,left_y,right_y,"gini")
                        #update best split if conditions are met
                        if curr_info_gain>max_info_gain:
                            best_split["feature_index"]=feature_index
                            best_split["threshold"]=threshold
                            best_split["dataset_left"]=dataset_left
                            best_split["dataset_right"]=dataset_right
                            best_split["info_gain"]=curr_info_gain
                            max_info_gain=curr_info_gain
            return best_split
        
    def split(self,dataset,feature_index,threshold):
        dataset_left=np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right=np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left,dataset_right
    
    def information_gain(self,parent,l_split,r_split,mode="gini"):
        weight_l=len(l_split)/len(parent)
        weight_r=len(r_split)/len(parent)
        if mode=="gini":
            gain=self.gini_index(parent)-(weight_l*self.gini_index(l_split)+ weight_r*self.gini_index(r_split))
        else:
            gain=self.entropy(parent)-(weight_l*self.entropy(l_split)+ weight_r*self.entropy(r_split))
        return gain
    
    def entropy(self,y):
        class_labels=np.unique(y)
        entropy=0
        for cls in class_labels:
            p_cls=len(y[y==cls])/len(y)
            entropy += -p_cls*np.log2(p_cls)
        return entropy
    def gini_index(self,y):
        class_labels=np.unique(y)
        gini=0
        for cls in class_labels:
            p_cls=len(y[y==cls])/len(y)
            gini += p_cls**2
        return 1-gini
    def calculate_leaf_value(self,Y):
        Y=list(Y)
        return max(Y, key=Y.count)
    def print_tree(self,tree=None,indent=" "):
        if not tree:
            tree=self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index),"<=", tree.threshold, "?",tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent+indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    def fit(self, X, Y):
        dataset= np.concatenate((X, Y), axis=1)
        self.root= self.build_tree(dataset)
    def predict(self,X):
        predictions= [self.make_prediction(x, self.root) for x in X]
        return predictions
    def make_prediction(self,x,tree):
        #print(tree.value)
        if tree.value!=None: return tree.value
        feature_val=x[tree.feature_index]
        #print(feature_val,tree.threshold,tree.feature_index)
        if feature_val<=tree.threshold:
            return self.make_prediction(x,tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [9]:
X2=train.iloc[:,:-1].values
Y2=train.iloc[:,-1].values.reshape(-1,1)
X2_train,X2_test,Y2_train,Y2_test= train_test_split(X2,Y2,test_size=.2,random_state=41)
#prev.drop('AGE',axis='columns',inplace=True)
X2_test

array([[ 2,  4,  2, ...,  1,  2,  1],
       [ 1, 12,  2, ...,  2,  2,  2],
       [ 1,  4,  2, ...,  2,  2,  2],
       ...,
       [ 1,  4,  1, ...,  1,  2,  2],
       [ 2,  4,  2, ...,  2,  2,  2],
       [ 2,  4,  2, ...,  2,  2,  2]], dtype=int64)

In [10]:
#prev.values
prev_  =  prev.values

In [16]:
tree_classifier=Decision_tree(min_samples_split=100, max_depth=12)
tree_classifier.fit(X2_train,Y2_train)
#tree_classifier.print_tree()
#y_pred_tree=tree_classifier.predict(prev_)


In [14]:
y_pred_tree=tree_classifier.predict(X2_test)
print("Precision:", precision_score(Y2_test, y_pred_tree, average='weighted'))
print("Recall:", recall_score(Y2_test, y_pred_tree, average='weighted'))
print("F1-score:", f1_score(Y2_test, y_pred_tree, average='weighted'))
#print(tree_classifier.root)

2 1 3
53 57 4
53 3 4
4 10 1
53 44 4
2 1 6
2 1 2
53 50 4
4 9 1
4 6 1
1 1 3
55 49 4
12 5 1
12 11 1
55 59 4
2 1 12
55 51 4
12 12 1
2 1 11
1 1 6
2 1 3
34 57 4
34 3 4
4 10 1
34 44 4
2 1 14
34 16 4
2 1 6
2 1 10
2 1 9
2 1 3
65 57 4
4 10 1
4 4 1
65 64 4
65 79 4
1 1 0
1 1 2
2 1 9
65 68 4
1 1 3
42 49 4
42 27 4
12 5 1
12 11 1
42 37 4
2 1 2
2 1 13
42 40 4
2 1 10
2 1 3
74 57 4
12 10 1
1 1 0
74 75 4
1 1 10
2 1 6
2 1 3
72 57 4
12 10 1
2 1 0
72 65 4
2 1 10
2 1 2
2 1 3
61 57 4
12 10 1
2 1 0
61 65 4
2 1 11
2 1 14
61 63 4
1 1 2
1 1 3
50 49 4
12 5 1
12 11 1
50 59 4
2 1 12
50 51 4
2 1 7
2 1 14
2 1 10
2 1 3
54 57 4
54 3 4
12 10 1
1 1 0
54 16 4
2 1 6
54 47 4
1 1 2
54 56 4
1 1 3
54 49 4
4 5 1
54 67 4
4 3 1
2 1 14
54 60 4
1 1 6
54 58 4
1 1 0
1 1 3
48 49 4
48 27 4
6 5 1
6 11 1
6 8 1
2 1 14
1 1 0
2 1 10
48 33 4
1 1 3
61 49 4
12 5 1
12 11 1
61 59 4
2 1 11
2 1 12
61 66 4
2 1 9
2 1 8
1 1 3
78 49 4
12 5 1
12 11 1
78 59 4
2 1 11
2 1 12
78 66 4
2 1 14
78 89 4
2 1 3
49 57 4
49 3 4
4 10 1
49 44 4
2 1 6
2 1 2
49 50 4
4 4

4 5 1
60 67 4
4 3 1
2 1 14
60 60 4
2 1 6
1 1 2
60 59 4
1 1 3
80 49 4
4 5 1
80 67 4
80 75 4
2 1 9
4 1 1
80 100 4
1 1 2
2 1 6
1 1 3
79 49 4
9 5 1
9 11 1
9 8 1
9 9 1
79 71 4
79 73 4
79 91 4
79 90 4
1 1 3
24 49 4
24 27 4
12 5 1
24 6 4
12 10 1
12 11 1
1 1 2
24 11 4
24 13 4
2 1 3
46 57 4
46 3 4
4 10 1
46 44 4
2 1 6
2 1 2
46 50 4
4 4 1
4 3 1
2 1 3
31 57 4
31 3 4
4 10 1
31 44 4
2 1 14
31 16 4
2 1 6
2 1 10
2 1 9
1 1 3
53 49 4
6 5 1
6 11 1
6 8 1
53 62 4
53 56 4
2 1 11
2 1 0
1 1 3
74 49 4
4 5 1
74 67 4
74 75 4
4 3 1
2 1 14
74 70 4
1 1 6
2 1 0
1 1 3
65 49 4
12 5 1
12 11 1
65 59 4
2 1 11
2 1 12
65 66 4
2 1 9
2 1 8
2 1 3
24 57 4
24 3 4
12 10 1
1 1 0
24 16 4
2 1 6
24 47 4
2 1 11
24 19 4
1 1 3
56 49 4
12 5 1
12 11 1
56 59 4
1 1 12
1 1 3
79 49 4
12 5 1
12 11 1
79 59 4
2 1 11
2 1 12
79 66 4
2 1 14
79 89 4
2 1 3
54 57 4
54 3 4
12 10 1
2 1 0
54 19 4
2 1 6
54 46 4
1 1 2
54 55 4
2 1 3
24 57 4
24 3 4
4 10 1
24 44 4
2 1 14
24 16 4
2 1 6
2 1 10
2 1 9
1 1 3
65 49 4
12 5 1
12 11 1
65 59 4
2 1 11
2 1 12
65 66 4
2

9 9 1
73 71 4
73 73 4
1 1 3
71 49 4
9 5 1
9 11 1
9 8 1
9 9 1
71 71 4
2 1 0
2 1 12
2 1 6
2 1 3
33 57 4
33 3 4
12 10 1
2 1 0
33 19 4
2 1 6
33 46 4
2 1 9
2 1 10
2 1 3
84 57 4
4 10 1
4 4 1
84 64 4
84 79 4
4 3 1
84 87 4
2 1 15
2 1 6
1 1 3
50 49 4
12 5 1
12 11 1
50 59 4
2 1 12
50 51 4
2 1 7
2 1 14
2 1 10
1 1 3
55 49 4
12 5 1
12 11 1
55 59 4
2 1 12
55 51 4
12 12 1
1 1 11
2 1 3
2 57 4
2 3 4
2 1 4
1 1 3
45 49 4
45 27 4
9 5 1
9 11 1
9 8 1
45 34 4
2 1 11
45 40 4
9 9 1
2 1 3
36 57 4
36 3 4
12 10 1
2 1 0
36 19 4
2 1 6
36 46 4
2 1 9
2 1 10
1 1 3
65 49 4
4 5 1
65 67 4
4 3 1
1 1 14
65 58 4
1 1 3
52 49 4
12 5 1
12 11 1
52 59 4
2 1 12
52 51 4
12 12 1
2 1 11
2 1 6
2 1 3
75 57 4
10 10 1
10 4 1
10 9 1
2 1 3
13 57 4
13 3 4
12 10 1
1 1 0
13 16 4
2 1 3
76 57 4
4 10 1
4 4 1
76 64 4
76 79 4
1 1 0
2 1 2
4 3 1
76 67 4
2 1 3
33 57 4
33 3 4
4 10 1
33 44 4
2 1 14
33 16 4
2 1 6
2 1 10
2 1 9
1 1 3
34 49 4
34 27 4
12 5 1
12 11 1
34 37 4
2 1 5
2 1 11
2 1 15
1 1 0
1 1 3
60 49 4
6 5 1
6 11 1
6 8 1
60 62 4
60 56 4
2 1 13
1

1 1 0
2 1 11
1 1 3
55 49 4
10 5 1
10 11 1
10 8 1
10 9 1
1 1 2
2 1 3
30 57 4
30 3 4
12 10 1
1 1 0
30 16 4
2 1 6
30 47 4
2 1 11
30 19 4
2 1 3
87 57 4
6 10 1
6 4 1
6 9 1
6 6 1
87 65 4
87 66 4
2 1 2
2 1 3
46 57 4
46 3 4
4 10 1
46 44 4
1 1 6
2 1 14
46 48 4
2 1 10
1 1 3
78 49 4
12 5 1
12 11 1
78 59 4
2 1 11
1 1 12
78 63 4
78 67 4
78 88 4
1 1 3
57 49 4
4 5 1
57 67 4
4 3 1
2 1 14
57 60 4
2 1 6
2 1 2
4 4 1
1 1 3
74 49 4
4 5 1
74 67 4
74 75 4
4 3 1
2 1 14
74 70 4
2 1 6
1 1 0
1 1 3
56 49 4
8 5 1
8 11 1
8 8 1
56 62 4
56 56 4
2 1 11
1 1 0
56 51 4
1 1 3
37 49 4
37 27 4
12 5 1
12 11 1
37 37 4
2 1 5
2 1 11
2 1 15
1 1 0
1 1 3
59 49 4
4 5 1
59 67 4
4 3 1
2 1 14
59 60 4
2 1 6
1 1 2
59 59 4
1 1 3
64 49 4
12 5 1
12 11 1
64 59 4
2 1 11
2 1 12
64 66 4
2 1 9
2 1 8
2 1 3
61 57 4
4 10 1
4 4 1
61 64 4
4 3 1
2 1 12
61 61 4
2 1 14
61 58 4
2 1 3
63 57 4
9 10 1
9 4 1
9 9 1
9 6 1
63 67 4
2 1 3
73 57 4
6 10 1
6 4 1
6 9 1
6 6 1
73 65 4
73 66 4
1 1 2
2 1 3
51 57 4
51 3 4
4 10 1
51 44 4
1 1 6
2 1 14
51 48 4
2 1 11
2 1 10

2 1 14
64 60 4
2 1 7
2 1 12
2 1 9
2 1 3
50 57 4
50 3 4
4 10 1
50 44 4
2 1 6
2 1 2
50 50 4
4 4 1
4 3 1
2 1 3
88 57 4
6 10 1
6 4 1
6 9 1
6 6 1
88 65 4
88 66 4
2 1 2
1 1 3
72 49 4
6 5 1
6 11 1
6 8 1
72 62 4
1 1 6
72 73 4
72 64 4
6 6 1
2 1 3
44 57 4
44 3 4
12 10 1
2 1 0
44 19 4
1 1 6
2 1 8
2 1 9
2 1 11
2 1 3
41 57 4
41 3 4
4 10 1
41 44 4
2 1 14
41 16 4
2 1 6
2 1 10
2 1 9
2 1 3
12 57 4
12 3 4
12 10 1
2 1 0
12 19 4
2 1 9
2 1 6
2 1 10
1 1 2
2 1 3
79 57 4
4 10 1
4 4 1
79 64 4
79 79 4
2 1 0
79 68 4
1 1 10
79 69 4
2 1 3
67 57 4
4 10 1
4 4 1
67 64 4
67 79 4
2 1 0
67 68 4
67 67 4
4 3 1
1 1 3
72 49 4
4 5 1
72 67 4
72 75 4
4 3 1
2 1 14
72 70 4
1 1 6
2 1 0
1 1 3
78 49 4
6 5 1
6 11 1
6 8 1
78 62 4
1 1 6
78 73 4
2 1 0
1 1 3
1 49 4
1 27 4
12 5 1
1 6 4
2 1 8
1 2 4
1 0 4
1 1 0
2 1 3
59 57 4
6 10 1
6 4 1
6 9 1
6 6 1
59 65 4
2 1 3
68 57 4
12 10 1
1 1 0
68 75 4
1 1 10
2 1 6
1 1 3
22 49 4
22 27 4
12 5 1
22 6 4
12 10 1
12 11 1
2 1 2
1 1 0
22 15 4
2 1 3
22 57 4
22 3 4
12 10 1
2 1 0
22 19 4
2 1 6
22 46 4
2 1 9
2

...

1 1 3
48 49 4
48 27 4
12 5 1
12 11 1
48 37 4
2 1 2
2 1 13
48 40 4
2 1 10
2 1 3
74 57 4
10 10 1
10 4 1
10 9 1
2 1 3
59 57 4
12 10 1
2 1 0
59 65 4
2 1 11
2 1 14
59 63 4
2 1 2
2 1 6
1 1 3
51 49 4
4 5 1
51 67 4
4 3 1
2 1 14
51 60 4
2 1 6
2 1 2
4 4 1
2 1 3
54 57 4
54 3 4
4 10 1
54 44 4
2 1 6
1 1 2
4 3 1
54 48 4
2 1 11
2 1 3
42 57 4
42 3 4
4 10 1
42 44 4
2 1 14
42 16 4
2 1 6
2 1 10
2 1 9
1 1 3
45 49 4
45 27 4
12 5 1
12 11 1
45 37 4
1 1 2
2 1 11
45 39 4
2 1 5
1 1 3
65 49 4
6 5 1
6 11 1
6 8 1
65 62 4
1 1 6
65 73 4
65 64 4
6 6 1
1 1 3
74 49 4
4 5 1
74 67 4
74 75 4
4 3 1
2 1 14
74 70 4
2 1 6
2 1 0
1 1 3
48 49 4
48 27 4
9 5 1
9 11 1
9 8 1
48 34 4
2 1 11
48 40 4
9 9 1
2 1 3
53 57 4
53 3 4
4 10 1
53 44 4
2 1 6
2 1 2
53 50 4
4 9 1
4 6 1
2 1 3
52 57 4
52 3 4
4 10 1
52 44 4
2 1 6
2 1 2
52 50 4
4 9 1
4 6 1
1 1 3
42 49 4
42 27 4
12 5 1
12 11 1
42 37 4
1 1 2
2 1 11
42 39 4
2 1 5
2 1 3
28 57 4
28 3 4
12 10 1
1 1 0
28 16 4
2 1 6
28 47 4
2 1 11
28 19 4
1 1 3
39 49 4
39 27 4
4 5 1
39 37 4
2 1 5
4 3 1
4 4 1
3

2 1 6
1 1 10
2 1 0
1 1 3
57 49 4
5 5 1
57 67 4
5 3 1
2 1 14
57 60 4
1 1 6
57 58 4
2 1 0
1 1 3
77 49 4
12 5 1
12 11 1
77 59 4
2 1 11
2 1 12
77 66 4
2 1 14
77 89 4
2 1 3
66 57 4
4 10 1
4 4 1
66 64 4
66 79 4
2 1 0
66 68 4
66 67 4
4 3 1
2 1 3
41 57 4
41 3 4
4 10 1
41 44 4
2 1 14
41 16 4
2 1 6
2 1 10
2 1 9
2 1 3
22 57 4
22 3 4
12 10 1
1 1 0
22 16 4
2 1 6
22 47 4
2 1 11
22 19 4
1 1 3
72 49 4
6 5 1
6 11 1
6 8 1
72 62 4
2 1 6
1 1 10
2 1 14
2 1 8
1 1 3
45 49 4
45 27 4
11 5 1
11 11 1
11 8 1
45 34 4
2 1 11
45 40 4
11 9 1
1 1 3
52 49 4
12 5 1
12 11 1
52 59 4
2 1 12
52 51 4
12 12 1
2 1 11
2 1 6
2 1 3
53 57 4
53 3 4
6 10 1
53 44 4
2 1 6
1 1 2
6 3 1
53 48 4
2 1 11
2 1 3
65 57 4
4 10 1
4 4 1
65 64 4
65 79 4
2 1 0
65 68 4
65 67 4
4 3 1
2 1 3
68 57 4
12 10 1
1 1 0
68 75 4
2 1 10
68 67 4
1 1 3
61 49 4
4 5 1
61 67 4
4 3 1
2 1 14
61 60 4
2 1 7
2 1 12
2 1 9
1 1 3
43 49 4
43 27 4
12 5 1
12 11 1
43 37 4
1 1 2
2 1 11
43 39 4
2 1 5
1 1 3
53 49 4
10 5 1
10 11 1
10 8 1
10 9 1
2 1 2
53 80 4
53 74 4
53 55 4
1 1 3
4

4 3 1
88 87 4
1 1 3
85 49 4
12 5 1
12 11 1
85 59 4
1 1 11
2 1 3
51 57 4
51 3 4
12 10 1
2 1 0
51 19 4
2 1 6
51 46 4
2 1 2
2 1 11
2 1 3
26 57 4
26 3 4
12 10 1
2 1 0
26 19 4
2 1 6
26 46 4
2 1 9
2 1 10
2 1 3
22 57 4
22 3 4
12 10 1
1 1 0
22 16 4
2 1 6
22 47 4
2 1 11
22 19 4
2 1 3
50 57 4
50 3 4
12 10 1
1 1 0
50 16 4
2 1 6
50 47 4
1 1 2
50 56 4
2 1 3
56 57 4
56 3 4
4 10 1
56 44 4
2 1 6
2 1 2
56 50 4
4 9 1
4 6 1
1 1 3
46 49 4
46 27 4
4 5 1
46 37 4
2 1 5
4 3 1
4 4 1
46 47 4
2 1 2
1 1 3
37 49 4
37 27 4
9 5 1
9 11 1
9 8 1
37 34 4
2 1 11
37 40 4
2 1 12
2 1 3
72 57 4
4 10 1
4 4 1
72 64 4
72 79 4
1 1 0
2 1 2
4 3 1
72 67 4
2 1 3
69 57 4
4 10 1
4 4 1
69 64 4
69 79 4
2 1 0
69 68 4
1 1 10
69 69 4
1 1 3
72 49 4
3 5 1
72 67 4
72 75 4
3 3 1
1 1 3
56 49 4
6 5 1
6 11 1
6 8 1
56 62 4
56 56 4
2 1 11
2 1 0
1 1 3
86 49 4
12 5 1
12 11 1
86 59 4
2 1 11
2 1 12
86 66 4
2 1 14
86 89 4
2 1 3
27 57 4
27 3 4
4 10 1
27 44 4
2 1 14
27 16 4
2 1 6
2 1 10
2 1 9
2 1 3
0 57 4
0 3 4
0 1 4
2 1 10
2 1 9
0 0 4
2 1 0
4 4 1
1 1 3
7

2 1 9
2 1 10
2 1 3
70 57 4
4 10 1
4 4 1
70 64 4
70 79 4
1 1 0
1 1 2
2 1 9
70 68 4
1 1 3
48 49 4
48 27 4
12 5 1
12 11 1
48 37 4
1 1 2
2 1 11
48 39 4
2 1 5
1 1 3
53 49 4
12 5 1
12 11 1
53 59 4
2 1 12
53 51 4
12 12 1
2 1 11
2 1 6
2 1 3
50 57 4
50 3 4
12 10 1
2 1 0
50 19 4
2 1 6
50 46 4
2 1 2
2 1 11
2 1 3
54 57 4
54 3 4
12 10 1
2 1 0
54 19 4
2 1 6
54 46 4
2 1 2
2 1 11
1 1 3
81 49 4
4 5 1
81 67 4
81 75 4
2 1 9
4 1 1
81 100 4
2 1 2
2 1 6
1 1 3
53 49 4
12 5 1
12 11 1
53 59 4
2 1 12
53 51 4
12 12 1
2 1 11
1 1 6
2 1 3
29 57 4
29 3 4
12 10 1
1 1 0
29 16 4
2 1 6
29 47 4
2 1 11
29 19 4
2 1 3
47 57 4
47 3 4
12 10 1
2 1 0
47 19 4
2 1 6
47 46 4
1 1 2
47 55 4
1 1 3
40 49 4
40 27 4
9 5 1
9 11 1
9 8 1
40 34 4
2 1 11
40 40 4
2 1 12
1 1 3
54 49 4
9 5 1
9 11 1
9 8 1
9 9 1
54 71 4
2 1 0
2 1 12
2 1 6
2 1 3
23 57 4
23 3 4
12 10 1
2 1 0
23 19 4
2 1 6
23 46 4
2 1 9
2 1 10
2 1 3
49 57 4
49 3 4
4 10 1
49 44 4
2 1 6
1 1 2
4 3 1
49 48 4
2 1 11
2 1 3
1 57 4
1 3 4
1 1 4
2 1 10
2 1 9
1 0 4
2 1 3
45 57 4
45 3 4
4 10 1


66 59 4
2 1 11
2 1 12
66 66 4
2 1 9
2 1 8
1 1 3
70 49 4
12 5 1
12 11 1
70 59 4
2 1 11
2 1 12
70 66 4
2 1 14
70 89 4
1 1 3
0 49 4
0 27 4
12 5 1
0 6 4
2 1 8
0 2 4
0 0 4
1 1 0
2 1 11
1 1 3
27 49 4
27 27 4
4 5 1
27 19 4
2 1 3
29 57 4
29 3 4
12 10 1
1 1 0
29 16 4
2 1 6
29 47 4
2 1 11
29 19 4
1 1 3
69 49 4
5 5 1
69 67 4
69 75 4
5 3 1
2 1 14
69 70 4
69 68 4
2 1 7
2 1 3
79 57 4
4 10 1
4 4 1
79 64 4
79 79 4
1 1 0
1 1 2
2 1 9
79 68 4
2 1 3
27 57 4
27 3 4
4 10 1
27 44 4
2 1 14
27 16 4
2 1 6
2 1 10
2 1 9
2 1 3
32 57 4
32 3 4
12 10 1
2 1 0
32 19 4
2 1 6
32 46 4
2 1 9
2 1 10
2 1 3
54 57 4
54 3 4
12 10 1
2 1 0
54 19 4
2 1 6
54 46 4
2 1 2
2 1 11
1 1 3
53 49 4
12 5 1
12 11 1
53 59 4
2 1 12
53 51 4
12 12 1
2 1 11
2 1 6
2 1 3
34 57 4
34 3 4
12 10 1
1 1 0
34 16 4
2 1 6
34 47 4
2 1 11
34 19 4
1 1 3
57 49 4
12 5 1
12 11 1
57 59 4
2 1 12
57 51 4
12 12 1
2 1 11
2 1 6
1 1 3
61 49 4
4 5 1
61 67 4
4 3 1
2 1 14
61 60 4
2 1 7
2 1 12
2 1 9
1 1 3
57 49 4
4 5 1
57 67 4
4 3 1
1 1 14
57 58 4
1 1 3
76 49 4
10 5 1
10 11 

In [10]:
pred2={
    'LABEL':y_pred_tree
}
submision=pd.DataFrame(pred2)
submision.to_csv("submision_tree3.csv",index="ID")

In [18]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=1)

tree.fit(X_train, y_train)

y_pred_tree= tree.predict(X_test)

#print("Accuracy:", accuracy_score(y_test, y_pred_tree))
print("Precision:", precision_score(y_test, y_pred_tree, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_tree, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred_tree, average='weighted'))

Precision: 0.6615065731757186
Recall: 0.662196228698195
F1-score: 0.6588096791543447


In [41]:
pred2={
    'LABEL':y_pred_tree
}
submision=pd.DataFrame(pred2)
submision.to_csv("submision_tree.csv",index="ID")

In [ ]:
#Make a seaborn heatmap